<a href="https://colab.research.google.com/github/douwvn/RebelwayAppliedMLDouw/blob/dataio/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
)

# New Section

In [ ]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
from logging import log
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu(x)
    return logits


model = Net().to(device)
print(model)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0

  with torch.inference_mode():
    for X,y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314102 [    0/60000]
loss: 0.578819 [ 6400/60000]
loss: 0.388078 [12800/60000]
loss: 0.506748 [19200/60000]
loss: 0.440992 [25600/60000]
loss: 0.433729 [32000/60000]
loss: 0.377252 [38400/60000]
loss: 0.534002 [44800/60000]
loss: 0.478383 [51200/60000]
loss: 0.523904 [57600/60000]
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.419956 

Epoch 2
-------------------------------
loss: 0.264720 [    0/60000]
loss: 0.364893 [ 6400/60000]
loss: 0.293614 [12800/60000]
loss: 0.397491 [19200/60000]
loss: 0.399600 [25600/60000]
loss: 0.384098 [32000/60000]
loss: 0.309855 [38400/60000]
loss: 0.504058 [44800/60000]
loss: 0.394539 [51200/60000]
loss: 0.467566 [57600/60000]
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.386897 

Epoch 3
-------------------------------
loss: 0.234711 [    0/60000]
loss: 0.325326 [ 6400/60000]
loss: 0.214922 [12800/60000]
loss: 0.308534 [19200/60000]
loss: 0.406549 [25600/60000]
loss: 0.360402 [32000/60000]
loss: 0.268562 [38400/

In [ ]:
torch.save(model.state_dict(), "fashion_mnist_base_model.pth")
print("Saved PyTorch Model State to fashion_mnist_base_model.pth")

Saved PyTorch Model State to fashion_mnist_base_model.pth


In [ ]:
model = Net().to(device)
model.load_state_dict(torch.load("fashion_mnist_base_model.pth"))
model.eval()

from PIL import Image

image_path = "1.jpg"
image = Image.open(image_path).convert("L")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28,28)),
])

image = transform(image).to(device)

with torch.inference_mode():
  output = model(image)
  prediction = torch.argmax(output).item()

print(f"Prediction: {prediction}")

Prediction: 4


In [ ]:
()